In [1]:
import pandas as pd
from sqlalchemy import create_engine
from pandas import DataFrame

In [2]:
!pip install psycopg2

In [3]:
csv_file = "governors_county_candidate.csv"
candidate_data_df = pd.read_csv(csv_file)
candidate_data_df

,state,county,candidate,party,votes,won
0,Delaware,Kent,John Carney,DEM,44352,True
1,Delaware,Kent,Julianne Murray,REP,39332,False
2,Delaware,Kent,Kathy DeMatteis,IPD,1115,False
3,Delaware,Kent,John Machurek,LIB,616,False
4,Delaware,New Castle,John Carney,DEM,191678,True
...,...,...,...,...,...,...
5068,West Virginia,Wood,Danny Lutz,MNP,377,False
5069,West Virginia,Wyoming,Jim Justice,REP,6941,True
5070,West Virginia,Wyoming,Ben Salango,DEM,1240,False
5071,West Virginia,Wyoming,Erika Kolenich,LIB,86,False


In [4]:

candidate_data_df.rename(columns={"state":"state_name"},inplace=True)
candidate_data_df.rename(columns={"county":"county_name"},inplace=True)
candidate_data_df

,state_name,county_name,candidate,party,votes,won
0,Delaware,Kent,John Carney,DEM,44352,True
1,Delaware,Kent,Julianne Murray,REP,39332,False
2,Delaware,Kent,Kathy DeMatteis,IPD,1115,False
3,Delaware,Kent,John Machurek,LIB,616,False
4,Delaware,New Castle,John Carney,DEM,191678,True
...,...,...,...,...,...,...
5068,West Virginia,Wood,Danny Lutz,MNP,377,False
5069,West Virginia,Wyoming,Jim Justice,REP,6941,True
5070,West Virginia,Wyoming,Ben Salango,DEM,1240,False
5071,West Virginia,Wyoming,Erika Kolenich,LIB,86,False


In [5]:
url = 'https://www.ssa.gov/international/coc-docs/states.html'
state_tables= pd.read_html(url)



In [6]:
state_abbr_df = state_tables[0]


In [7]:
state_abbr_df.columns = ['state_name', 'state_code']
state_abbr_df['state_name'] = state_abbr_df['state_name'].str.capitalize() 
state_abbr_df

,state_name,state_code
0,Alabama,AL
1,Alaska,AK
2,American samoa,AS
3,Arizona,AZ
4,Arkansas,AR
5,California,CA
6,Colorado,CO
7,Connecticut,CT
8,Delaware,DE
9,District of columbia,DC


In [8]:
merge_df = pd.merge(state_abbr_df,candidate_data_df, on="state_name")
merge_df.dropna()
merge_df

,state_name,state_code,county_name,candidate,party,votes,won
0,Delaware,DE,Kent,John Carney,DEM,44352,True
1,Delaware,DE,Kent,Julianne Murray,REP,39332,False
2,Delaware,DE,Kent,Kathy DeMatteis,IPD,1115,False
3,Delaware,DE,Kent,John Machurek,LIB,616,False
4,Delaware,DE,New Castle,John Carney,DEM,191678,True
...,...,...,...,...,...,...,...
3325,Washington,WA,Whatcom,Loren Culp,REP,55445,False
3326,Washington,WA,Whitman,Jay Inslee,DEM,10766,True
3327,Washington,WA,Whitman,Loren Culp,REP,10030,False
3328,Washington,WA,Yakima,Loren Culp,REP,54074,True


In [9]:
rds_connection_string = "postgres:rootroot@localhost:5432/ETLproject_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
engine.table_names()

['state_abbr', 'election_demographic', 'governors_county', 'gov_cty_candidate']

In [12]:
candidate_data_df

,state_name,county_name,candidate,party,votes,won
0,Delaware,Kent,John Carney,DEM,44352,True
1,Delaware,Kent,Julianne Murray,REP,39332,False
2,Delaware,Kent,Kathy DeMatteis,IPD,1115,False
3,Delaware,Kent,John Machurek,LIB,616,False
4,Delaware,New Castle,John Carney,DEM,191678,True
...,...,...,...,...,...,...
5068,West Virginia,Wood,Danny Lutz,MNP,377,False
5069,West Virginia,Wyoming,Jim Justice,REP,6941,True
5070,West Virginia,Wyoming,Ben Salango,DEM,1240,False
5071,West Virginia,Wyoming,Erika Kolenich,LIB,86,False


In [11]:
merge_df.to_sql(name='gov_cty_candidate', con=engine, if_exists='append', index=False)

In [12]:
pd.read_sql_query('select * from gov_cty_candidate', con=engine).head()

,id,state_code,state_name,county_name,candidate,party,votes,won
0,1,DE,Delaware,Kent,John Carney,DEM,44352,True
1,2,DE,Delaware,Kent,Julianne Murray,REP,39332,False
2,3,DE,Delaware,Kent,Kathy DeMatteis,IPD,1115,False
3,4,DE,Delaware,Kent,John Machurek,LIB,616,False
4,5,DE,Delaware,New Castle,John Carney,DEM,191678,True
